In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\unsupervised-pre-training-vgg'

In [9]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from IPython import display
import matplotlib.gridspec as gridspec
from tensorflow_src import Autoencoder
from tensorflow_src.utils import visualize_preds
from tensorflow_src.evaluation import load_ae

In [4]:
BATCH_SIZE = 64
EPOCHS = 10
H,W,C = 256,256,3
lr = 1e-5
latent_dim = 512

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_batches = datagen.flow_from_directory(
    '/kaggle/input/skin-cancer-preprocessed-nusaifa/data/DA',
    target_size=(H,W),
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder training
    shuffle=True
)

In [ ]:
test_batches = datagen.flow_from_directory(
    '/kaggle/input/skin-cancer9-classesisic/',
    target_size=(H,W),
    batch_size=BATCH_SIZE,
    class_mode='input',  # Use 'input' mode for autoencoder training
    shuffle=True
)

In [7]:
autoencoder = Autoencoder((256,256,3))
model = autoencoder.build_autoencoder()

In [8]:
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

model.compile(optimizer=Adam(lr), loss=ssim_loss)

In [ ]:
checkpoint_filepath = '/kaggle/working/ae_model_256.keras'
cbacks = [
    tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
]

# Training

In [ ]:
history = model.fit(train_batches, steps_per_epoch=len(total_images) // BATCH_SIZE, epochs=25, callbacks=cbacks, validation_data=test_batches)

# Evaluation

In [ ]:
saved_model = load_ae(checkpoint_filepath)

In [ ]:
batch_images, _ = next(test_batches)
predictions = autoencoder_model.predict(batch_images)

In [ ]:
visualize_preds(batch_images, predictions)